In [1]:
!nvidia-smi

Fri Dec  1 10:20:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.216.04   Driver Version: 450.216.04   CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-80GB      On   | 00000000:B7:00.0 Off |                    0 |
| N/A   39C    P0    63W / 400W |      0MiB / 81252MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# !pip install --upgrade \
#     accelerate==0.22.0 \
#     bitsandbytes==0.41.0 \
#     transformers==4.34.0 \
#     sentencepiece

# # peft==0.5.0
# !pip install --upgrade peft --no-dependencies

In [1]:
import os
import re
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig
)
from peft import PeftModel, PeftConfig
from tqdm import tqdm
from copy import deepcopy

[2023-12-19 01:56:20,243] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [3]:
BASE_DIR = '/home/jovyan/tsukanov/test_calc/lora_train/lora-saiga/models'
BASE_MODEL = 'Open-Orca/Mistral-7B-OpenOrca'
BASE_LORA = f'{BASE_DIR}/saiga_mistal_7b_lora'
CHECKPOINT_NAME = 'v1/checkpoint-455'
CHECKPOINT = f'{BASE_DIR}/{CHECKPOINT_NAME}'

config = PeftConfig.from_pretrained(CHECKPOINT)
assert config.base_model_name_or_path == BASE_MODEL
assert config == PeftConfig.from_pretrained(BASE_LORA)

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True, llm_int8_threshold=4.0, llm_int8_enable_fp32_cpu_offload=False,
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map=device,
    load_in_8bit=False,
    # load_in_8bit=True,
    # quantization_config=quantization_config,
)
model = PeftModel.from_pretrained(
    model,
    CHECKPOINT,
    torch_dtype=torch.float16,
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(BASE_LORA, use_fast=False)
generation_config = GenerationConfig.from_pretrained(BASE_LORA)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}

In [5]:
generation_config = GenerationConfig(**{
    "pad_token_id": 0,
    "bos_token_id": 1,
    "eos_token_id": 2,
    "max_new_tokens": 1000,
    "no_repeat_ngram_size": 15,
    "repetition_penalty": 1.01,

    "greedy": True,
    "do_sample": False,

    # "do_sample": True,
    # "temperature": 0.1,
    # "top_k": 10,
    # "top_p": 0.95
})
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "greedy": true,
  "max_new_tokens": 1000,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.01
}

In [17]:
tokenizer.bos_token

'<s>'

In [6]:
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()

In [7]:
"""
Helpers to support streaming generate output.
Borrowed from https://github.com/oobabooga/text-generation-webui/blob/ad37f396fc8bcbab90e11ecf17c56c97bfbd4a9c/modules/callbacks.py
"""

import gc
import traceback
from queue import Queue
from threading import Thread
from transformers import StoppingCriteria, StoppingCriteriaList

class Stream(StoppingCriteria):
    def __init__(self, callback_func=None):
        self.callback_func = callback_func

    def __call__(self, input_ids, scores) -> bool:
        if self.callback_func is not None:
            self.callback_func(input_ids[0])
        return False


class Iteratorize:

    """
    Transforms a function that takes a callback
    into a lazy iterator (generator).
    """

    def __init__(self, func, kwargs={}, callback=None):
        self.mfunc = func
        self.c_callback = callback
        self.q = Queue()
        self.sentinel = object()
        self.kwargs = kwargs
        self.stop_now = False

        def _callback(val):
            if self.stop_now:
                raise ValueError
            self.q.put(val)

        def gentask():
            try:
                ret = self.mfunc(callback=_callback, **self.kwargs)
            except ValueError:
                pass
            except:
                traceback.print_exc()
                pass

            self.q.put(self.sentinel)
            if self.c_callback:
                self.c_callback(ret)

        self.thread = Thread(target=gentask)
        self.thread.start()

    def __iter__(self):
        return self

    def __next__(self):
        obj = self.q.get(True, None)
        if obj is self.sentinel:
            raise StopIteration
        else:
            return obj

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.stop_now = True

In [8]:
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt").to(model.device)
    generate_params = {
        **data,
        "generation_config": generation_config,
    }
    input_ids_length = generate_params['input_ids'].shape[1]

    # Stream the reply 1 token at a time.
    # This is based on the trick of using 'stopping_criteria' to create an iterator,
    # from https://github.com/oobabooga/text-generation-webui/blob/ad37f396fc8bcbab90e11ecf17c56c97bfbd4a9c/modules/text_generation.py#L216-L243.

    def generate_with_callback(callback=None, **kwargs):
        if "stopping_criteria" in kwargs:
            del kwargs["stopping_criteria"]
        kwargs.setdefault(
            "stopping_criteria", StoppingCriteriaList()
        )
        kwargs["stopping_criteria"].append(
            Stream(callback_func=callback)
        )
        with torch.no_grad():
            model.generate(**kwargs)

    def generate_with_streaming(**kwargs):
        return Iteratorize(
            generate_with_callback, kwargs, callback=None
        )

    def get_calc_result(expression):
        try:
            result = str(round(eval(expression), 2))
            if '.' in result:
                left, right = result.split('.', 1)
                if set(right) == {'0'}:
                    result = left
            # print(expression, '=', result)
            return result
        except Exception as ex:
            print(expression, ex)
            return None

    is_end = False
    for _ in range(100):
        if is_end:
            break
        with generate_with_streaming(**generate_params) as generator:
            for output in generator:
                output = output[input_ids_length:]
                if output[-1] == tokenizer.eos_token_id or \
                        len(output) == generate_params['generation_config'].max_new_tokens - 1:
                    is_end = True
                    yield decoded_output
                    break
                decoded_output = tokenizer.decode(output)
                if '</calculator>' in decoded_output:
                    exression = decoded_output.rsplit('</calculator>', 1)[0]
                    if '<calculator>' in exression:
                        exression = exression.rsplit('<calculator>', 1)[1].strip()
                        result = get_calc_result(exression)
                        if result is not None:
                            decoded_output += result
                            prompt += decoded_output
                            # print(f'-----{prompt}-----')
                            data = tokenizer(prompt, return_tensors="pt").to(model.device)
                            new_gen_config = deepcopy(generate_params['generation_config'])
                            new_gen_config.max_new_tokens -= len(output)
                            generate_params = {
                                **data,
                                "generation_config": new_gen_config,
                            }
                            input_ids_length = generate_params['input_ids'].shape[1]
                            yield decoded_output
                            break


def simple_generate(model, tokenizer, prompt, generation_config):
    model.eval()
    data = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output_ids = model.generate(
            **data,
            generation_config=generation_config
        )[0][len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids)
    return output.strip()

In [9]:
def generate_ft(model, tokenizer, prompt, generation_config):
    model.eval()
    data = tokenizer(prompt, return_tensors="pt").to(model.device)
    generate_params = {
        **data,
        "generation_config": generation_config,
    }

    # Stream the reply 1 token at a time.
    # This is based on the trick of using 'stopping_criteria' to create an iterator,
    # from https://github.com/oobabooga/text-generation-webui/blob/ad37f396fc8bcbab90e11ecf17c56c97bfbd4a9c/modules/text_generation.py#L216-L243.

    all_tokens = list(range(len(tokenizer)))
    next_tokens = []

    def prefix_allowed_tokens_fn(batch_id, input_ids):
        if len(next_tokens) > 0:
            allowed_tokens = [next_tokens.pop(0)]
            # print(allowed_tokens)
            return allowed_tokens
        return all_tokens

    def generate_with_callback(callback=None, **kwargs):
        if "stopping_criteria" in kwargs:
            del kwargs["stopping_criteria"]
        kwargs.setdefault(
            "stopping_criteria", StoppingCriteriaList()
        )
        kwargs["stopping_criteria"].append(
            Stream(callback_func=callback)
        )
        with torch.no_grad():
            model.generate(
                prefix_allowed_tokens_fn=prefix_allowed_tokens_fn,
                **kwargs
            )

    def generate_with_streaming(**kwargs):
        return Iteratorize(
            generate_with_callback, kwargs, callback=None
        )

    def get_calc_result(expression):
        try:
            expression = re.sub(r'[^0-9+\-*/.()]', '', expression.replace(',', '.'))
            result = str(round(eval(expression), 2))
            if '.' in result:
                left, right = result.split('.', 1)
                if set(right) == {'0'}:
                    result = left
            # print(expression, '=', result)
            return result
        except Exception as ex:
            print(expression, ex)
            return None

    is_end = False
    last_calculated_idx = len(prompt)
    calc_start, calc_end = '<calculator>', '</calculator>'
    # calc_start, calc_end = '[[', ']]'

    for _ in range(100):
        if is_end:
            break
        with generate_with_streaming(**generate_params) as generator:
            for output in generator:
                if output[-1] == tokenizer.eos_token_id or \
                        len(output) == generate_params['generation_config'].max_new_tokens - 1:
                    is_end = True
                    yield decoded_output
                    break
                decoded_output = tokenizer.decode(output, skip_special_tokens=True)
                new_output = decoded_output[last_calculated_idx:]
                # print('new_output:', last_calculated_idx, new_output)
                if calc_end in new_output:
                    exression = new_output.rsplit(calc_end, 1)[0]
                    if calc_start in exression:
                        exression = exression.rsplit(calc_start, 1)[1].strip()
                        result = get_calc_result(exression)
                        if result is not None:
                            result_tokens = tokenizer(result, add_special_tokens=False)['input_ids']
                            # print(result, result_tokens)
                            if result_tokens[0] == 28705:
                                # NOTE: There might also be a token for the minus sign
                                result_tokens = result_tokens[1:]
                            next_tokens.extend(result_tokens)
                            yield decoded_output
                    last_calculated_idx = len(decoded_output)

In [10]:
def get_few_shot_examples():
    yield (
        """Семья из 12 обезьян собрала 10 куч бананов. В 6 кучах было 9 рук, по 14 бананов в каждой руке, \
а в остальных кучах было 12 рук, по 9 бананов в каждой руке. Сколько бананов достанется каждой обезьяне, \
если они поделят бананы поровну между собой?""",
        """В первых 6 связках было 6 * 9 * 14 = <calculator>6 * 9 * 14</calculator>756 бананов.
Осталось 10 - 6 = <calculator>10 - 6</calculator>4 пучка.
В 4 оставшихся связках было 4 * 12 * 9 = <calculator>4 * 12 * 9</calculator>432 банана.
Всего было 756 + 432 = <calculator>756 + 432</calculator>1188 бананов.
Каждая обезьяна получит 1188 / 12 = <calculator>1188 / 12</calculator>99 бананов.
#### 99""",
    )
    yield (
        """Джек оказывается на необитаемом острове. Он хочет немного соли, чтобы приправить рыбу. Он набирает 2 литра \
морской воды в старое ведро. Если вода содержит 20 % соли, сколько мл соли получит Джек, когда вся вода испарится?""",
        """Сначала найдем, сколько литров соли содердится в морской воде: 2 литра * 20% = \
<calculator>2 * 20 * 0.01</calculator>0.4 литра.
Затем умножим это количество на 1000 мл/литр, чтобы найти количество миллилитров соли, которое получит Джек: \
0.4 литра * 1000 мл/литр = <calculator>0.4 * 1000</calculator>400 мл.
#### 400""",
    )
    yield (
        """Бетти копит деньги на новый бумажник, который стоит 100 долларов. У Бетти есть только половина денег, \
которые ей нужны. Ее родители решили дать ей на эти цели 15 долларов, а бабушка и дедушка — в два раза больше, \
чем ее родители. Сколько еще денег нужно Бетти, чтобы купить бумажник?""",
        """Вначале у Бетти было всего $100 / 2 = $<calculator>100 / 2</calculator>50.
Бабушка и дедушка Бетти дали ей $15 * 2 = $<calculator>15 * 2</calculator>30.
Это означает, что Бетти нужно еще $100 - $50 - $30 - $15 = $<calculator>100 - 50 - 30 - 15</calculator>5.
#### 5""",
    )
#     yield (
#         """Чтобы приготовить пиццу вместе с другими ингредиентами, Кимберу нужно 10 чашек воды, \
# 16 чашек муки и в 1/2 раза больше чайных ложек соли, чем чашек муки. Подсчитайте общее количество \
# чашек воды, муки и чайных ложек соли, которые ей понадобятся для приготовления пиццы.""",
#         """Чтобы приготовить пиццу, Кимбер потратила вдвое меньше чайных ложек соли, чем чашек муки, \
# то есть ей нужно 1 / 2 * 16 = <calculator>16 * 1 / 2</calculator>8 чайных ложек соли.
# Общее количество чашек муки и чайных ложек соли, которое ей нужно, равно 8 + 16 = <calculator>8 + 16</calculator>24.
# Ей также нужно 10 чашек воды, что означает, что общее количество чашек воды, муки и чайных ложек соли, \
# которые ей нужны, составляет 24 + 10 = <calculator>24 + 10</calculator>34.
# #### 34""",
#     )
#     yield (
#         """Каролин занимается игрой на фортепиано по 20 минут в день, а на скрипке — в три раза дольше. \
# Если она тренируется шесть дней в неделю, сколько минут она тратит на занятия в месяц из четырех недель?""",
#         """Сначала найдем общее время занятий Кэролайн на скрипке, утроив ее время занятий на фортепиано: \
# 20 минут в день * 3 = <calculator>20 * 3</calculator>60 минут в день.
# Затем найдите общее количество времени, которое она тратит на занятия каждый день: 60 минут/день + \
# 20 минут/день = <calculator>60 + 20</calculator>80 минут/день.
# Затем найдите общее время, которое она тратит на тренировки каждую неделю: 80 минут в день * \
# 6 дней в неделю = <calculator>80 * 6</calculator>480 минут в неделю.
# Затем найдите общее время, которое она тратит на занятия каждый месяц: 480 минут в неделю * \
# 4 недели в месяц = <calculator>480 * 4</calculator>1920 минут в месяц.
# #### 1920""",
#     )


def make_prompt(tokenizer, inp, system_prompt=DEFAULT_SYSTEM_PROMPT):
    conversation = Conversation(system_prompt=system_prompt)
    # for question, answer in get_few_shot_examples():
    #     conversation.add_user_message(question)
    #     conversation.add_bot_message(answer)
    conversation.add_user_message(inp)
    # return conversation.messages
    prompt = conversation.get_prompt(tokenizer)
    return prompt

In [11]:
gsm_test = pd.read_csv('gsm_test.csv')

actual_answers = [
    float(a.rsplit('#### ', 1)[1].strip())
    for a in gsm_test['answer']
]

def get_answer(a):
    if 'Ответ:' not in a:
        # print(a, '\n', '=' * 50, '\n')
        print('|', end='')
    numbers = re.findall(r'(\d+\.\d+|\d+\,\d+|\d+)', a)
    if len(numbers) == 0:
        return float('inf')
    return float(numbers[-1].replace(',', '.'))

### Fine-tuned evaluation

In [37]:
system_prompt = "Ты — русскоязычный автоматический ассистент. Твоя задача: внимательно отвечать на вопросы пользователя, расписывая решение по шагам и выполняя одно арифметическое действие за раз. Тебе необходимо пользоваться калькулятором вместо того, чтобы считать результат арифметического выражения самостоятельно. Для этого помести математическое выражение между тэгами \"<calculator>\" и \"</calculator>\", чтобы вызвать функцию калькулятора и узнать ответ. Используй \"+\" для сложения, \"-\" для вычитания, \"*\" для умножения, \"/\" для деления чисел. Финальный ответ напиши в самом конце в поле \"Ответ:\". Начнем!\n\n### Задача:\n{instruction}\n\n### Решение:\n"

inputs = [
    "Если Анне 9 лет, а ее брат вдвое старше ее, сколько лет будет ее брату через 3 года?",
    "Доставка пиццы Эшли стоит 15 долларов. Какова общая сумма, которую Эшли должна дать доставщику, если она хочет дать чаевые, равные 1/5 заказанной суммы?"
]
for inp in inputs:
    prompt = system_prompt.format(instruction=inp)

    # output = simple_generate(model, tokenizer, prompt, generation_config)
    # print(prompt, output, sep='')

    result = ''
    for output in generate_ft(model, tokenizer, prompt, generation_config):
        # print(output)
        result = output
    print(result)

    print("\n==============================")

Ты — русскоязычный автоматический ассистент. Твоя задача: внимательно отвечать на вопросы пользователя, расписывая решение по шагам и выполняя одно арифметическое действие за раз. Тебе необходимо пользоваться калькулятором вместо того, чтобы считать результат арифметического выражения самостоятельно. Для этого помести математическое выражение между тэгами "<calculator>" и "</calculator>", чтобы вызвать функцию калькулятора и узнать ответ. Используй "+" для сложения, "-" для вычитания, "*" для умножения, "/" для деления чисел. Финальный ответ напиши в самом конце в поле "Ответ:". Начнем!

### Задача:
Если Анне 9 лет, а ее брат вдвое старше ее, сколько лет будет ее брату через 3 года?

### Решение:
Ее брату 9 * 2 = <calculator>9 * 2</calculator>18 лет.
Через 3 года ее брату будет 18 + 3 = <calculator>18 + 3</calculator>21 год.
Ответ: 21

Ты — русскоязычный автоматический ассистент. Твоя задача: внимательно отвечать на вопросы пользователя, расписывая решение по шагам и выполняя одно ариф

In [38]:
print(system_prompt)

Ты — русскоязычный автоматический ассистент. Твоя задача: внимательно отвечать на вопросы пользователя, расписывая решение по шагам и выполняя одно арифметическое действие за раз. Тебе необходимо пользоваться калькулятором вместо того, чтобы считать результат арифметического выражения самостоятельно. Для этого помести математическое выражение между тэгами "<calculator>" и "</calculator>", чтобы вызвать функцию калькулятора и узнать ответ. Используй "+" для сложения, "-" для вычитания, "*" для умножения, "/" для деления чисел. Финальный ответ напиши в самом конце в поле "Ответ:". Начнем!

### Задача:
{instruction}

### Решение:



In [12]:
gen_dir = f'./saiga/{CHECKPOINT_NAME}'
os.makedirs(gen_dir, exist_ok=True)
gen_dir

'./saiga/v1/checkpoint-455'

In [40]:
generated_outputs = []
with open(f'{gen_dir}/gen_ft_output.txt', 'a') as f_out:
    for i, d in tqdm(enumerate(gsm_test.to_dict('records'))):
        if i < len(generated_outputs):
            continue
        prompt = system_prompt.format(instruction=d['question'])
        result = ''
        for output in generate_ft(model, tokenizer, prompt, generation_config):
            result = output
        generated_outputs.append(result.strip())
        f_out.write(f'\n### OUTPUT {i} ###\n{output}\n')
        f_out.flush()

276it [48:36,  8.67s/it]

In [14]:
# with open(f'{gen_dir}/gen_ft_output.txt', 'r') as f_in:
#     generated_outputs = re.split(r'\n### OUTPUT \d+ ###\n', f_in.read())[1:]

len(generated_outputs)

400

In [ ]:
import pickle

with open(f'{gen_dir}/gen_ft_answers.pkl', 'wb') as f:
    pickle.dump(generated_outputs, f)

# with open(f'{gen_dir}/gen_ft_answers.pkl', 'rb') as f:
#     generated_outputs = pickle.load(f)

In [117]:
# 364
generated_answers = [get_answer(a) for a in generated_outputs]
np.isclose(generated_answers, actual_answers, rtol=1e-9, atol=1e-3).mean()

0.475

In [15]:
# 455
generated_answers = [get_answer(a) for a in generated_outputs]
np.isclose(generated_answers, actual_answers, rtol=1e-9, atol=1e-3).mean()

0.51

In [104]:
# 546
generated_answers = [get_answer(a) for a in generated_outputs]
np.isclose(generated_answers, actual_answers, rtol=1e-9, atol=1e-3).mean()

0.505

In [27]:
# 637
generated_answers = [get_answer(a) for a in generated_outputs]
np.isclose(generated_answers, actual_answers, rtol=1e-9, atol=1e-3).mean()

0.515

In [10]:
# 728
generated_answers = [get_answer(a) for a in generated_outputs]
np.isclose(generated_answers, actual_answers, rtol=1e-9, atol=1e-3).mean()

0.5

### 3-shot evaluation

In [10]:
# system_prompt = 'Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.'
# Результат выражения будет вычислен при помощи калькулятора и записан сразу же после вызова (после закрывающего тэга). \
system_prompt = """\
Ты — русскоязычный автоматический ассистент. \
Твоя задача: внимательно отвечать на вопросы пользователя, расписывая решение по шагам и выполняя одно арифметическое действие за раз. \
Тебе необходимо пользоваться калькулятором вместо того, чтобы считать результат арифметического выражения самостоятельно. \
Для этого помести математическое выражение между тэгами "<calculator>" и "</calculator>", чтобы вызвать функцию калькулятора и узнать ответ. \
Используй "+" для сложения, "-" для вычитания, "*" для умножения, "/" для деления чисел. \
Финальный результат напиши в самом конце после "####".

Приведу несколько примеров ответов на вопросы – внимательно следуй этим примерам:

"""

system_prompt += '\n\n'.join(map(lambda q_a: f'Вопрос: {q_a[0]}\nОтвет: {q_a[1]}', get_few_shot_examples()))
system_prompt += """

ВНИМАНИЕ:
1) Oбязательно делай вызовы функции calculator вида "<calculator>выражение</calculator>" каждый раз, когда тебе нужно получить результат математического выражения (после знака = )!
2) Не забудь в конце ответа написать результат.
"""
system_prompt = system_prompt.replace('####', 'Результат:')
# print(system_prompt)

inputs = [
    "Если Анне 9 лет, а ее брат вдвое старше ее, сколько лет будет ее брату через 3 года?",
    "Доставка пиццы Эшли стоит 15 долларов. Какова общая сумма, которую Эшли должна дать доставщику, если она хочет дать чаевые, равные 1/5 заказанной суммы?"
]
for inp in inputs:
    prompt = make_prompt(tokenizer, inp, system_prompt)
    # print(prompt)
    # output = simple_generate(model, tokenizer, prompt, generation_config)
    result = ''
    for output in generate(model, tokenizer, prompt, generation_config):
        # print(output)
        result += output
    print(result)
    print("\n==============================")

Ты — русскоязычный автоматический ассистент. Твоя задача: внимательно отвечать на вопросы пользователя, расписывая решение по шагам и выполняя одно арифметическое действие за раз. Тебе необходимо пользоваться калькулятором вместо того, чтобы считать результат арифметического выражения самостоятельно. Для этого помести математическое выражение между тэгами "<calculator>" и "</calculator>", чтобы вызвать функцию калькулятора и узнать ответ. Используй "+" для сложения, "-" для вычитания, "*" для умножения, "/" для деления чисел. Финальный ответ напиши в самом конце в поле "Ответ:". Начнем!

### Задача:
{instruction}

### Решение:



In [14]:
print(system_prompt)

Ты — русскоязычный автоматический ассистент. Твоя задача: внимательно отвечать на вопросы пользователя, расписывая решение по шагам и выполняя одно арифметическое действие за раз. Тебе необходимо пользоваться калькулятором вместо того, чтобы считать результат арифметического выражения самостоятельно. Для этого помести математическое выражение между тэгами "<calculator>" и "</calculator>", чтобы вызвать функцию калькулятора и узнать ответ. Используй "+" для сложения, "-" для вычитания, "*" для умножения, "/" для деления чисел. Финальный результат напиши в самом конце после "Результат:".

Приведу несколько примеров ответов на вопросы – внимательно следуй этим примерам:

Вопрос: Семья из 12 обезьян собрала 10 куч бананов. В 6 кучах было 9 рук, по 14 бананов в каждой руке, а в остальных кучах было 12 рук, по 9 бананов в каждой руке. Сколько бананов достанется каждой обезьяне, если они поделят бананы поровну между собой?
Ответ: В первых 6 связках было 6 * 9 * 14 = <calculator>6 * 9 * 14</ca

In [47]:
# generated_outputs = []
with open('gen_3shot_output.txt', 'a') as f_out:
    for i, d in tqdm(enumerate(gsm_test.to_dict('records'))):
        if i < len(generated_outputs):
            continue
        prompt = make_prompt(tokenizer, d['question'], system_prompt)
        result = []
        for output in generate(model, tokenizer, prompt, generation_config):
            result.append(output)
        output = ''.join(result).strip()
        generated_outputs.append(output)
        f_out.write(f'\n### OUTPUT {i} ###\n{output}\n')
        f_out.flush()

400it [25:45,  3.86s/it]


In [48]:
len(generated_outputs)

400

In [49]:
import pickle

# with open('gen_3shot.pkl', 'wb') as f:
#     pickle.dump(generated_outputs, f)

# with open('gen_3shot.pkl', 'rb') as f:
#     generated_outputs = pickle.load(f)

In [50]:
generated_answers = [get_answer(a) for a in generated_outputs]
np.isclose(generated_answers, actual_answers, rtol=1e-15, atol=1e-5).mean()

|||||||||||||||||||||||||||||||

0.39

### 3-shot evaluation without calculator call

In [51]:
system_prompt = """\
Ты — русскоязычный автоматический ассистент. \
Твоя задача: внимательно отвечать на вопросы пользователя, расписывая решение по шагам и выполняя одно арифметическое действие за раз. \
Тебе необходимо пользоваться калькулятором вместо того, чтобы считать результат арифметического выражения самостоятельно. \
Для этого помести математическое выражение между тэгами "<calculator>" и "</calculator>", чтобы вызвать функцию калькулятора и узнать ответ. \
Используй "+" для сложения, "-" для вычитания, "*" для умножения, "/" для деления чисел. \
Финальный результат напиши в самом конце после "####".

Приведу несколько примеров ответов на вопросы – внимательно следуй этим примерам:

"""

system_prompt += '\n\n'.join(map(lambda q_a: f'Вопрос: {q_a[0]}\nОтвет: {q_a[1]}', get_few_shot_examples()))
system_prompt += """

ВНИМАНИЕ:
1) Oбязательно делай вызовы функции calculator вида "<calculator>выражение</calculator>" каждый раз, когда тебе нужно получить результат математического выражения (после знака = )!
2) Не забудь в конце ответа написать результат.
"""
system_prompt = system_prompt.replace('####', 'Результат:')
# print(system_prompt)

inputs = [
    "Если Анне 9 лет, а ее брат вдвое старше ее, сколько лет будет ее брату через 3 года?",
    "Доставка пиццы Эшли стоит 15 долларов. Какова общая сумма, которую Эшли должна дать доставщику, если она хочет дать чаевые, равные 1/5 заказанной суммы?"
]
for inp in inputs:
    prompt = make_prompt(tokenizer, inp, system_prompt)
    # print(prompt)
    output = simple_generate(model, tokenizer, prompt, generation_config)
    print(output)
    print("\n==============================")

Вначале найдем возраст брата Анны: 9 * 2 = <calculator>9 * 2</calculator>18.
Затем добавим 3 года к возрасту брата: 18 + 3 = <calculator>18 + 3</calculator>21.
Результат: 21</s>

Вначале найдем общую сумму заказа: 15 долларов + 1/5 * 15 долларов = <calculator>15 + 15 * 0.2</calculator>21 доллара.
Теперь найдем сумму чаевых: 1/5 * 21 долларов = <calculator>21 * 0.2</calculator>4.2 доллара.
Общая сумма, которую Эшли должна дасть доставщику, равна 21 доллара + 4.2 доллара = <calculator>21 + 4.2</calculator>25.2 доллара.
Результат: 25.2</s>



In [52]:
print(system_prompt)

Ты — русскоязычный автоматический ассистент. Твоя задача: внимательно отвечать на вопросы пользователя, расписывая решение по шагам и выполняя одно арифметическое действие за раз. Тебе необходимо пользоваться калькулятором вместо того, чтобы считать результат арифметического выражения самостоятельно. Для этого помести математическое выражение между тэгами "<calculator>" и "</calculator>", чтобы вызвать функцию калькулятора и узнать ответ. Используй "+" для сложения, "-" для вычитания, "*" для умножения, "/" для деления чисел. Финальный результат напиши в самом конце после "Результат:".

Приведу несколько примеров ответов на вопросы – внимательно следуй этим примерам:

Вопрос: Семья из 12 обезьян собрала 10 куч бананов. В 6 кучах было 9 рук, по 14 бананов в каждой руке, а в остальных кучах было 12 рук, по 9 бананов в каждой руке. Сколько бананов достанется каждой обезьяне, если они поделят бананы поровну между собой?
Ответ: В первых 6 связках было 6 * 9 * 14 = <calculator>6 * 9 * 14</ca

In [53]:
generated_outputs_no_calc = []
with open('gen_3shot_no_calc_output.txt', 'a') as f_out:
    for i, d in tqdm(enumerate(gsm_test.to_dict('records'))):
        if i < len(generated_outputs_no_calc):
            continue
        prompt = make_prompt(tokenizer, d['question'], system_prompt)
        output = simple_generate(model, tokenizer, prompt, generation_config)
        generated_outputs_no_calc.append(output)
        f_out.write(f'\n### OUTPUT {i} ###\n{output}\n')
        f_out.flush()

400it [1:00:55,  9.14s/it]


In [54]:
len(generated_outputs_no_calc)

400

In [55]:
generated_answers = [get_answer(a) for a in generated_outputs_no_calc]
np.isclose(generated_answers, actual_answers, rtol=1e-15, atol=1e-5).mean()

||||||||||||||||||||||||

0.3425

In [56]:
import pickle

with open('gen_3shot_no_calc.pkl', 'wb') as f:
    pickle.dump(generated_outputs_no_calc, f)

# with open('gen_3shot_no_calc.pkl', 'rb') as f:
#     generated_outputs_no_calc = pickle.load(f)

### 0-shot evaluation without calculator call

In [20]:
# system_prompt = 'Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.'
# Результат выражения будет вычислен при помощи калькулятора и записан сразу же после вызова (после закрывающего тэга). \
system_prompt = """\
Ты — русскоязычный автоматический ассистент. \
Твоя задача: внимательно отвечать на вопросы пользователя, расписывая решение по шагам и выполняя одно арифметическое действие за раз. \
Финальный результат напиши в виде единственного числа в самом конце после "Результат:".
"""

inputs = [
    "Если Анне 9 лет, а ее брат вдвое старше ее, сколько лет будет ее брату через 3 года?",
    "Доставка пиццы Эшли стоит 15 долларов. Какова общая сумма, которую Эшли должна дать доставщику, если она хочет дать чаевые, равные 1/5 заказанной суммы?"
]
for inp in inputs:
    prompt = make_prompt(tokenizer, inp, system_prompt)
    # print(prompt)
    output = simple_generate(model, tokenizer, prompt, generation_config)
    print(output)
    print("\n==============================")

1. Найдем возраст брата: 9 (Анна) * 2 = 18 лет.
2. Добавим 3 года к возрасту брата: 18 + 3 = 21 лет.

Результат: 21 лет.</s>

Чтобы найти общую сумму, которую Эшли должна дать доставщику с учетом чаевых, мы должны сначала найти размер чаевых.

Шаг 1: Найдите размер чаевых, равные 1/5 заказанной сумма.
Размер заказа = 15 долларов
Чаевые = 1/5 * 15 = 3 доллара

Шаг 2: Добавьте размер чаевых к заказанной сумме.
Общая сумма = 15 + 3 = 18 долларов

Результат: 18 долларов</s>



In [21]:
print(system_prompt)

Ты — русскоязычный автоматический ассистент. Твоя задача: внимательно отвечать на вопросы пользователя, расписывая решение по шагам и выполняя одно арифметическое действие за раз. Финальный результат напиши в виде единственного числа в самом конце после "Результат:".



In [23]:
generated_outputs_zero = []
with open('gen_0shot_output.txt', 'w') as f_out:
    for i, d in tqdm(enumerate(gsm_test.to_dict('records'))):
        prompt = make_prompt(tokenizer, d['question'], system_prompt)
        output = simple_generate(model, tokenizer, prompt, generation_config)
        generated_outputs_zero.append(output)
        f_out.write(f'\n### OUTPUT {i} ###\n{output}\n')
        f_out.flush()

400it [57:53,  8.68s/it]


In [24]:
len(generated_outputs_zero)

400

In [25]:
import pickle

# with open('gen_0shot.pkl', 'wb') as f:
#     pickle.dump(generated_outputs_zero, f)

# with open('gen_0shot.pkl', 'rb') as f:
#     generated_outputs_zero = pickle.load(f)

In [53]:
generated_answers = [get_answer(a) for a in generated_outputs_zero]
np.isclose(generated_answers, actual_answers, rtol=1e-15, atol=1e-5).mean()

||||||||||

0.4275